In [16]:
import torch
from torch.utils.data import Dataset
import h5py
import numpy as np
import bisect
import sys
from pickle import FALSE
from collections import OrderedDict
from tqdm import tqdm


sys.path.append('..')  # Add the parent directory to the sys.path

import utils.dataset_loader as dataset_loader

! gpustat

galileo2       Thu Sep 19 19:47:20 2024  535.183.01
[0] NVIDIA A40 | 18°C,   0 % |     0 / 49140 MB |


In [13]:
# split the windows to fixed size context and label windows
fs = 250
context_window_size = 9*60*fs  # minutes * seconds * fs
label_window_size = 1*60*fs  # minutes * seconds * fs
window_size = context_window_size+label_window_size


# split_pSignal_file = '/mnt/qnap/liranc6/data/with_R_beats/icentia11k-continuous-ecg_normal_sinus_subset_npArrays_splits/10minutes_window.h5'  # rambo
split_pSignal_file = "/home/liranc6/data/with_R_beats/icentia11k-continuous-ecg_normal_sinus_subset_npArrays_splits/10minutes/train/p0_to_p32.h5"  # newton

# Instantiate the class
dataset = dataset_loader.SingleLeadECGDatasetCrops_SSSD(context_window_size, label_window_size, split_pSignal_file, start_patiant=3, end_patiant=6, data_with_RR=True, return_with_RR=True)

# Loop over the dataset
# print(f'{len(dataset)=}')
for i in range(len(dataset)):
    # print(i)
    context, label = dataset.__getitem__(i)
    print(f"{context.shape=}, {label.shape=}")
    # Perform any assertions or checks here
    if i >9:
        break

key='00003'
key='00004'
key='00005'
key='00006'
context.shape=(2, 135000), label.shape=(2, 15000)
context.shape=(2, 135000), label.shape=(2, 15000)
context.shape=(2, 135000), label.shape=(2, 15000)
context.shape=(2, 135000), label.shape=(2, 15000)
context.shape=(2, 135000), label.shape=(2, 15000)
context.shape=(2, 135000), label.shape=(2, 15000)
context.shape=(2, 135000), label.shape=(2, 15000)
context.shape=(2, 135000), label.shape=(2, 15000)
context.shape=(2, 135000), label.shape=(2, 15000)
context.shape=(2, 135000), label.shape=(2, 15000)
context.shape=(2, 135000), label.shape=(2, 15000)


In [31]:
import numpy as np

# Create a 3D array with shape (2, 3, 4)
data = np.array([[[ 1,  2,  3,  4],
                  [ 5,  6,  7,  8],
                  [ 9, 10, 11, 12]],

                 [[13, 14, 15, 16],
                  [17, 18, 19, 20],
                  [21, 22, 23, 24]]])

class Args:
    class Data:
        target = 2

args = Args()
args.data = Args.Data()

# Select the column specified by args.data.target
data = data[:, [args.data.target]]

print("\nSelected column:")
print(data)

# Calculate the mean along axis 0 (column-wise)
train_mean = np.mean(data, axis=0)

# Calculate the standard deviation along axis 0 (column-wise)
train_std = np.std(data, axis=0)

print("\nMean of train_data along axis 0:")
print(train_mean)

print("\nStandard deviation of train_data along axis 0:")
print(train_std)


Selected column:
[[[ 9 10 11 12]]

 [[21 22 23 24]]]

Mean of train_data along axis 0:
[[15. 16. 17. 18.]]

Standard deviation of train_data along axis 0:
[[6. 6. 6. 6.]]


In [1]:
split_pSignal_file

NameError: name 'split_pSignal_file' is not defined

In [27]:
def _get_statistics_for_normalization(filename):
    mean = 0
    std = 0
    with h5py.File(filename, 'r') as h5_file:
        keys = list(h5_file.keys())
        num_datasets = len(keys)
        
        #use tqdm to show progress bar
        pbar_keys = tqdm(keys, total=num_datasets)
        for key, i in enumerate(pbar_keys):
            pbar_keys.set_description(f"Reading {key}")
            data = h5_file[key][()][:, 0, :]
            mean += np.mean(data, axis=0)
            std += np.std(data, axis=0)
            
            if i >10:
                break
            
        mean /= len(keys)
        std /= len(keys)
    return mean, std

mean, std = _get_statistics_for_normalization(split_pSignal_file)
print(f"{mean=}, {std=}")        

Reading 00000:   0%|          | 0/32 [00:00<?, ?it/s]

Reading 00032: 100%|██████████| 32/32 [00:49<00:00,  1.56s/it]

mean=array([0.09622615, 0.08381402, 0.04730365, ..., 0.00225265, 0.0024922 ,
       0.00269125]), std=array([0.32326099, 0.29838442, 0.23312771, ..., 0.09183643, 0.09137844,
       0.09245316])


In [25]:
def show_first_n_samples(dataset, n):
    context = dataset[:n][:]
        
def get_first_dataset_from_file(filename):
    with h5py.File(filename, 'r') as h5_file:
        key = list(h5_file.keys())[0]
        data = h5_file[key][()]
    return data

data = get_first_dataset_from_file(split_pSignal_file)
print(f"{data.shape=}")
show_first_n_samples(data, 1)

data.shape=(165, 2, 150000)
